In [13]:
import openai, os, requests
from IPython.display import display, HTML, Markdown, JSON, Latex


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"


# Azure OpenAI setup
openai.api_base = "xxxxx" # Add your endpoint here
openai.api_key = os.getenv("OPENAI_API_AZURE_KEY_REWILD") # Add your OpenAI API key here
deployment_id = "GPT4" # Add your deployment ID here

# Azure Cognitive Search setup
search_endpoint = "xxxxxx" # Add your Azure Cognitive Search endpoint here
search_key = os.getenv("AZURE_AI_SEARCH_RESOURCE_KEY_REWILD") # Add your Azure Cognitive Search admin key here
search_index_name = "plant-data-3-index" # Add your Azure Cognitive Search index name here





In [70]:
def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
 
    :param deployment_id: The deployment ID for the model to use with your own data.
 
    To remove this configuration, simply set openai.requestssession to None.
    """
 
    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):
 
        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)
 
    session = requests.Session()
 
    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )
    openai.requestssession = session
 


prompt="""Get me three plants that best match these requirements: plant_type:(tree) AND plant_size:(>5m) AND flower_color:(orange)      
  
If three options are not available for the requested criteria give me the closest matches. If multiple matches are found then prioritise the selected choices using the highest count value.     
  
Provide me the scientific name, common name and flower colour of the plants in a JSON object. Do not provide anything other than the JSON object.
"""

completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": prompt,},
              {"role":"system", "content": "You are a native plant expert helping Australian suburban gardeners transform their gardens with local native plant recommendations. Use your knowledge base to provide recommendations and information"}
             ],
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
                "stream": "true",
                "stop": "null",
                "temperature": "0",
                "top_p": "1",
                "engine": "gpt-4",
                "topNDocuments": 20,
                "strictness": 2
                
            }
        }
    ]
)
print(completion)


{
  "id": "3ea062a6-e868-46a2-8b3a-025beb7bdb66",
  "model": "gpt-4",
  "created": 1700831398,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "```json\n[\n    {\n        \"scientific_name\": \"Corymbia ficifolia\",\n        \"common_name\": \"Red-flowering Gum\",\n        \"flower_color\": \"Orange Red\"\n    },\n    {\n        \"scientific_name\": \"Grevillea robusta\",\n        \"common_name\": \"Silky Oak\",\n        \"flower_color\": \"Golden Orange\"\n    },\n    {\n        \"scientific_name\": \"Tristaniopsis laurina\",\n        \"common_name\": \"Water Gum\",\n        \"flower_color\": \"Orange Yellow\"\n    }\n]\n```",
        "end_turn": true,
        "context": {
          "messages": [
            {
              "role": "tool",
              "content": "{\"citations\": [{\"content\": \"https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/C

In [71]:
# Extract the text content from the 'completion' object
# Adjust the key access according to the actual structure of your 'completion' object
text_content = completion["choices"][0]["message"]["content"]

# Define the start and end markers for the "Description" section
# Ensure that these markers exactly match the text in the 'content'
start_marker = "```json\n"
end_marker = "\n```"

# Extract the section
start_index = text_content.find(start_marker)
end_index = text_content.find(end_marker)

if start_index != -1 and end_index != -1:
    # Adjust the start index to include the length of the start marker
    extracted_section = text_content[start_index + len(start_marker):end_index].strip()
else:
    extracted_section = "Section not found"

display(Markdown(extracted_section))

[
    {
        "scientific_name": "Corymbia ficifolia",
        "common_name": "Red-flowering Gum",
        "flower_color": "Orange Red"
    },
    {
        "scientific_name": "Grevillea robusta",
        "common_name": "Silky Oak",
        "flower_color": "Golden Orange"
    },
    {
        "scientific_name": "Tristaniopsis laurina",
        "common_name": "Water Gum",
        "flower_color": "Orange Yellow"
    }
]

In [66]:
prompt="""Provide me a summary of Corymbia ficifolia expressed in a way that can be understood by a 12 year old./
The format should be as follows delimited in symbols and presented using markdown with the heading as described below.
***
[Insert the common name here] (h2) 
Scientific name: (h3) 
Description: (h3) 
Requirements: (h3) 
Care: (h3)
Fun Factions: (h3)
^^^
"""

completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": prompt }],
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
                "stream": "true",
                "stop": "null",
                "temperature": "0",
                "top_p": "1",
                "engine": "gpt-4",
            }
        }
    ]
)
print(completion)

{
  "id": "52e85b85-dec1-4702-a309-26c29f2d7246",
  "model": "gpt-4",
  "created": 1700831307,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "***\n\n## Red-flowering Gum \n### Scientific name: \nCorymbia ficifolia\n\n### Description: \nThis tree is special because it's only found in the wild in the south-west of Western Australia. It's not very tall, and it's famous for its bright red, pink, or orange flowers that look really pretty. It also has a kind of fruit that looks like a little urn.\n\n### Requirements: \nThe Red-flowering Gum loves lots of sunlight and grows fast. It's also tough enough to handle windy places and salty air.\n\n### Care: \nIt's a tree that's easy to take care of once it's planted in a sunny spot. It can grow to be more than 5 meters tall, so make sure it has enough room to grow up!\n\n### Fun Facts: \nEven though it's called a gum

In [67]:
import json

# Extract the text content from the 'completion' object
# Adjust the key access according to the actual structure of your 'completion' object
text_content = completion["choices"][0]["message"]["content"]

# Define the start and end markers for the "Description" section
# Ensure that these markers exactly match the text in the 'content'
start_marker = "***"
end_marker = "^^^"

# Extract the section
start_index = text_content.find(start_marker)
end_index = text_content.find(end_marker)

if start_index != -1 and end_index != -1:
    # Adjust the start index to include the length of the start marker
    extracted_section = text_content[start_index + len(start_marker):end_index].strip()
else:
    extracted_section = "Section not found"

display(Markdown(extracted_section))


## Red-flowering Gum 
### Scientific name: 
Corymbia ficifolia

### Description: 
This tree is special because it's only found in the wild in the south-west of Western Australia. It's not very tall, and it's famous for its bright red, pink, or orange flowers that look really pretty. It also has a kind of fruit that looks like a little urn.

### Requirements: 
The Red-flowering Gum loves lots of sunlight and grows fast. It's also tough enough to handle windy places and salty air.

### Care: 
It's a tree that's easy to take care of once it's planted in a sunny spot. It can grow to be more than 5 meters tall, so make sure it has enough room to grow up!

### Fun Facts: 
Even though it's called a gum tree, it doesn't have the kind of gum you chew! It's also really popular for planting in gardens because it looks so beautiful when it flowers[doc1].